In [1]:
import pandas as pd
from datetime import datetime, timedelta
import random

# 您提供的公司名称列表
company_names = [
    "湖北国贸能源化工有限公司",
    "湖北国贸金属矿产有限公司",
    "湖北国贸汽车有限公司",
    "湖北国际贸易集团有限公司",
    # ... 添加所有公司名称
]

# 假定的行业列表
industries = ["煤炭", "钢材", "农产品", "汽车", "能源化工"]

# 初始化一个空列表来存储数据
data = []

# 模拟2023年的数据
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 8, 26)

# 计算总交易次数（每家公司每月平均4次交易）
total_transactions = len(company_names) * 4 * 12

for _ in range(total_transactions):
    # 随机选择公司名称
    company = random.choice(company_names)
    # 随机生成交易日期
    transaction_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    # 随机选择行业
    industry = random.choice(industries)
    # 随机生成下游客户名称
    customer_name = f"客户{random.randint(1000, 9999)}"
    # 随机生成金额
    SALES = round(random.uniform(10000, 5000000), 1)

    # 将生成的数据添加到列表
    data.append({
        "company_name": company,
        "customer_name": customer_name,
        "SALES": SALES,
        "date": transaction_date.strftime("%Y-%m-%d"),
        "industry": industry
    })
    

# 使用列表构造DataFrame
sales_df = pd.DataFrame(data)
sales_df['date'] = pd.to_datetime(sales_df['date'])
sales_df['SALES'] = pd.to_numeric(sales_df['SALES'])
# 打印DataFrame的前几行
print(sales_df.head())

   company_name customer_name      SALES       date industry
0    湖北国贸汽车有限公司        客户7440  2883073.7 2023-01-26      农产品
1  湖北国际贸易集团有限公司        客户7339  2977591.4 2023-12-13       汽车
2  湖北国际贸易集团有限公司        客户2118  2009631.8 2023-07-08       汽车
3  湖北国贸能源化工有限公司        客户1575  4624249.4 2023-08-11       汽车
4    湖北国贸汽车有限公司        客户2036  3135974.9 2023-10-10       煤炭


In [2]:
import sys
sys.path.append('..') 
from query import SalesQueryHelper
from entity import PerformanceIndicator,PerformanceQuerySchema


helper = SalesQueryHelper(sales_df)

# 创建查询参数
params = PerformanceQuerySchema(
    indicator='SALES',
    aggregation='YEAR',
    start_time='2023-01-01',
    end_time='2023-12-31',
    scope='GROUP',
    operator='>',
    value='100000000',
    company_name="None"  # 如果scope为GROUP，则可能不需要company_name
)

# 执行查询
result = helper.execute_query(params)
print(result)

Empty DataFrame
Columns: [company_name, year, SALES]
Index: []


/home/yepeng/ChatBI/chat-bi/src/notebook/../query/query_helper.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['year'] = filtered_df['date'].dt.year


In [3]:
params_by_month=PerformanceQuerySchema(
    indicator='SALES',
    aggregation='YEAR',
    start_time='2023-01-01',
    end_time='2023-12-31',
    scope='COMPANY',
    company_name="湖北国贸能源化工有限公司"  # 如果scope为GROUP，则可能不需要company_name
)
result = helper.execute_query(params_by_month)
print(result)

   company_name  year       SALES
0  湖北国贸能源化工有限公司  2023  73889240.7


In [4]:
import csv
import json

# 存储 Result 列的数据
results = []

# 打开 CSV 文件并读取内容
with open('../test/output.csv', 'r', encoding='utf-8') as csvfile:
    csvreader = csv.DictReader(csvfile)
    
    # 遍历每一行，提取 Result 列的数据
    for row in csvreader:
        results.append(row['Result'].replace("'", '"'))

# 打印 Result 列的数据以验证结果
data_results=[]
for query in results:
    try:
        # json_data = json.loads(query)
        params=PerformanceQuerySchema(**eval(query))
        if params.indicator == "SALES":
            print(params)
            data = helper.execute_query(params)
            data_results.append(data)
            print(data)
    except Exception as e:
        print(e)


indicator='SALES' aggregation='YEAR' start_time='2024-01-01' end_time='2024-12-31' scope='GROUP' sort_type='DESC' operator=None value=None company_name=None
   company_name  year       SALES
0    湖北国贸汽车有限公司  2024  46037361.6
1  湖北国贸能源化工有限公司  2024  52737264.5
2  湖北国贸金属矿产有限公司  2024  32396757.1
3  湖北国际贸易集团有限公司  2024  58433198.0
indicator='SALES' aggregation='MONTH' start_time='2024-01-01' end_time='2024-06-30' scope='GROUP' sort_type='DESC' operator='>' value='10000000' company_name='company_name_not_found'
    company_name    month       SALES
7   湖北国贸能源化工有限公司  2024-04  13220159.3
17  湖北国际贸易集团有限公司  2024-03  10383501.7
20  湖北国际贸易集团有限公司  2024-06  13157615.5
indicator='SALES' aggregation='YEAR' start_time='2023-01-01' end_time='2023-12-31' scope='GROUP' sort_type='DESC' operator=None value=None company_name=None
   company_name  year       SALES
0    湖北国贸汽车有限公司  2023  78594468.4
1  湖北国贸能源化工有限公司  2023  73889240.7
2  湖北国贸金属矿产有限公司  2023  62539313.4
3  湖北国际贸易集团有限公司  2023  77121542.6
indicator='

/home/yepeng/ChatBI/chat-bi/src/notebook/../query/query_helper.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['year'] = filtered_df['date'].dt.year
/home/yepeng/ChatBI/chat-bi/src/notebook/../query/query_helper.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['month'] = filtered_df['date'].dt.to_period('M')
/home/yepeng/ChatBI/chat-bi/src/notebook/../query/query_helper.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

   company_name customer_name      SALES       date industry
0    湖北国贸汽车有限公司        客户7440  2883073.7 2023-01-26      农产品
1  湖北国际贸易集团有限公司        客户7339  2977591.4 2023-12-13       汽车
2  湖北国际贸易集团有限公司        客户2118  2009631.8 2023-07-08       汽车
3  湖北国贸能源化工有限公司        客户1575  4624249.4 2023-08-11       汽车
4    湖北国贸汽车有限公司        客户2036  3135974.9 2023-10-10       煤炭


In [5]:
data_results

[   company_name  year       SALES
 0    湖北国贸汽车有限公司  2024  46037361.6
 1  湖北国贸能源化工有限公司  2024  52737264.5
 2  湖北国贸金属矿产有限公司  2024  32396757.1
 3  湖北国际贸易集团有限公司  2024  58433198.0,
     company_name    month       SALES
 7   湖北国贸能源化工有限公司  2024-04  13220159.3
 17  湖北国际贸易集团有限公司  2024-03  10383501.7
 20  湖北国际贸易集团有限公司  2024-06  13157615.5,
    company_name  year       SALES
 0    湖北国贸汽车有限公司  2023  78594468.4
 1  湖北国贸能源化工有限公司  2023  73889240.7
 2  湖北国贸金属矿产有限公司  2023  62539313.4
 3  湖北国际贸易集团有限公司  2023  77121542.6,
     company_name    month       SALES
 0     湖北国贸汽车有限公司  2024-01   8461945.8
 1     湖北国贸汽车有限公司  2024-03   6086437.3
 2     湖北国贸汽车有限公司  2024-04   6283523.5
 3     湖北国贸汽车有限公司  2024-05   6301694.7
 4   湖北国贸能源化工有限公司  2024-01   4007756.7
 5   湖北国贸能源化工有限公司  2024-02   7301502.6
 6   湖北国贸能源化工有限公司  2024-03   3782798.4
 7   湖北国贸能源化工有限公司  2024-04  13220159.3
 8   湖北国贸能源化工有限公司  2024-05   9822126.3
 9   湖北国贸能源化工有限公司  2024-06   4550879.6
 10  湖北国贸金属矿产有限公司  2024-01   3428979.7
 11  湖北国贸金属矿产有限公司  2024